In [9]:
import torch
import numpy as np
z = torch.from_numpy(np.array([[1,2,3],[4,5,6]]))
z.shape

torch.Size([2, 3])

In [12]:
torch.max(z,1)[0]

tensor([3, 6])

In [5]:
z.unsqueeze(dim = 1).shape

torch.Size([2, 1, 3])

In [8]:
tensor = torch.from_numpy(np.array([0, 1, 2, 3]))
mask = torch.from_numpy(np.array([True, False, True, False]))
tensor[mask]

tensor([0, 2])

In [33]:
o = torch.from_numpy(np.ones(3))
# z = np.zeros(3)
# v = np.stack([o,z])
# pc = torch.from_numpy(np.repeat(v, 53760, axis = 0).reshape((128,140,3)))

In [34]:
a = torch.from_numpy(np.array([True, False, True]))
o[a]

tensor([1., 1.], dtype=torch.float64)

In [29]:
myarr[0,1,:]

array([1., 1., 1.])

In [ ]:
pc = torch.from()

In [ ]:
def pointcloud2voxels3d_fast(cfg, pc, rgb):  # [B,N,3]
    vox_size = cfg.vox_size
    if cfg.vox_size_z != -1:
        vox_size_z = cfg.vox_size_z
    else:
        vox_size_z = vox_size

    batch_size = pc.shape[0]
    num_points = tf.shape(pc)[1]

    has_rgb = rgb is not None

    grid_size = 1.0
    half_size = grid_size / 2

    filter_outliers = True
    valid = tf.logical_and(pc >= -half_size, pc <= half_size)
    valid = tf.reduce_all(valid, axis=-1)

    vox_size_tf = tf.constant([[[vox_size_z, vox_size, vox_size]]], dtype=tf.float32)
    pc_grid = (pc + half_size) * (vox_size_tf - 1)
    indices_floor = tf.floor(pc_grid)
    indices_int = tf.cast(indices_floor, tf.int32)
    batch_indices = tf.range(0, batch_size, 1)
    batch_indices = tf.expand_dims(batch_indices, -1)
    batch_indices = tf.tile(batch_indices, [1, num_points])
    batch_indices = tf.expand_dims(batch_indices, -1)

    indices = tf.concat([batch_indices, indices_int], axis=2)
    indices = tf.reshape(indices, [-1, 4])

    r = pc_grid - indices_floor  # fractional part
    rr = [1.0 - r, r]

    if filter_outliers:
        valid = tf.reshape(valid, [-1])
        indices = tf.boolean_mask(indices, valid)

    def interpolate_scatter3d(pos):
        updates_raw = rr[pos[0]][:, :, 0] * rr[pos[1]][:, :, 1] * rr[pos[2]][:, :, 2]
        updates = tf.reshape(updates_raw, [-1])
        if filter_outliers:
            updates = tf.boolean_mask(updates, valid)

        indices_loc = indices
        indices_shift = tf.constant([[0] + pos])
        num_updates = tf.shape(indices_loc)[0]
        indices_shift = tf.tile(indices_shift, [num_updates, 1])
        indices_loc = indices_loc + indices_shift

        voxels = tf.scatter_nd(indices_loc, updates, [batch_size, vox_size_z, vox_size, vox_size])
        if has_rgb:
            if cfg.pc_rgb_stop_points_gradient:
                updates_raw = tf.stop_gradient(updates_raw)
            updates_rgb = tf.expand_dims(updates_raw, axis=-1) * rgb
            updates_rgb = tf.reshape(updates_rgb, [-1, 3])
            if filter_outliers:
                updates_rgb = tf.boolean_mask(updates_rgb, valid)
            voxels_rgb = tf.scatter_nd(indices_loc, updates_rgb, [batch_size, vox_size_z, vox_size, vox_size, 3])
        else:
            voxels_rgb = None

        return voxels, voxels_rgb

    voxels = []
    voxels_rgb = []
    for k in range(2):
        for j in range(2):
            for i in range(2):
                vx, vx_rgb = interpolate_scatter3d([k, j, i])
                voxels.append(vx)
                voxels_rgb.append(vx_rgb)

    voxels = tf.add_n(voxels)
    voxels_rgb = tf.add_n(voxels_rgb) if has_rgb else None

    return voxels, voxels_rgb


In [ ]:
def pointcloud2voxels3d_fast(cfg, pc, rgb):  # [B,N,3]
    vox_size = cfg.vox_size
    if cfg.vox_size_z != -1:
        vox_size_z = cfg.vox_size_z
    else:
        vox_size_z = vox_size

    batch_size = pc.shape[0]
    num_points = pc.shape[1]

    has_rgb = rgb is not None

    grid_size = 1.0
    half_size = grid_size / 2

    filter_outliers = True
    valid = (pc >= -half_size) * (pc <= half_size)
    valid = torch.prod(valid, dim=-1)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    vox_size_tf = torch.from_numpy(np.array([[[vox_size_z, vox_size, vox_size]]])).to(device)
    pc_grid = (pc + half_size) * (vox_size_tf - 1)
    indices_floor = torch.floor(pc_grid)
    indices_int = indices_floor.long()
    batch_indices = torch.arange(0, batch_size, 1)
    batch_indices = batch_indices.reshape(batch_indices.shape[0],1)
    batch_indices = batch_indices.repeat(1,num_points)
    batch_indices = batch_indices.reshape(batch_indices.shape[0], batch_indices.shape[1],1).to(device)
    indices = torch.cat((batch_indices, indices_int), dim=2)
    indices = indices.reshape(-1,4)
    r = pc_grid - indices_floor  # fractional part
    rr = [1.0 - r, r]

    if filter_outliers:
        valid = valid.reshape(-1)
        indices = indices[valid]

    def interpolate_scatter3d(pos,voxels):
        updates_raw = rr[pos[0]][:, :, 0] * rr[pos[1]][:, :, 1] * rr[pos[2]][:, :, 2]
        updates = updates_raw.reshape(-1)

        if filter_outliers:
            updates = updates[valid]

        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        indices_loc = indices
        indices_shift = torch.from_numpy(np.array([[0] + pos])).to(device)
        num_updates = indices_loc.shape[0]
        indices_shift = indices_shift.repeat(num_updates,1)
        indices_loc = indices_loc + indices_shift
        voxels[indices_loc[:,0],indices_loc[:,1],indices_loc[:,2],indices_loc[:,3]] += updates
        if has_rgb:
            if cfg.pc_rgb_stop_points_gradient:
                updates_raw = updates_raw.detach()
            updates_rgb = updates_raw. tf.expand_dims(updates_raw, axis=-1) * rgb
            updates_rgb = updates_rgb.reshape(-1,3)
            if filter_outliers:
                updates_rgb = updates_rgb[valid]
            voxels_rgb = torch.zeros((batch_size, vox_size_z, vox_size, vox_size), dtype=torch.float64)
            voxels_rgb[indices_loc[:,0],indices_loc[:,1],indices_loc[:,2],indices_loc[:,3]] = updates_rgb
        else:
            voxels_rgb = None

        return voxels, voxels_rgb
    import time
    t0 = time.perf_counter()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    voxels = torch.zeros((batch_size, vox_size_z, vox_size, vox_size),dtype=torch.float64).to(device)
    voxels_rgb = []
    for k in range(2):
        for j in range(2):
            for i in range(2):
                vx, vx_rgb = interpolate_scatter3d([k, j, i],voxels)
                voxels_rgb.append(vx_rgb)
    t1 =time.perf_counter()
    #print('Voxel_time {}'.format(t1-t0))
    voxels_rgb = torch.sum(torch.stack(voxels_rgb),0) if has_rgb else None

    return voxels, voxels_rgb